In [ ]:
!pip install -q langchain openai faiss-cpu sentence-transformers pymupdf torch

In [ ]:
import fitz  # PyMuPDF for PDF reading
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

ImportError: cannot import name 'Tensor' from 'torch' (unknown location)

In [ ]:


# Step 1: Load PDF text
def load_pdf_text(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            text += page.get_text()
    return text

# Load PDF content
pdf_path = "path/to/your_document.pdf"
document_text = load_pdf_text(pdf_path)
print("PDF loaded.")

# Step 2: Split document into chunks for embedding
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(document_text)

# Step 3: Create embeddings using Sentence-Transformer
class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name: str):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return self.model.encode(texts, show_progress_bar=True)

    def embed_query(self, text):
        return self.model.encode(text, show_progress_bar=False)

# Load Sentence-Transformer model for embeddings
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
sentence_embedding_model = SentenceTransformerEmbeddings(model_name=embedding_model_name)

# Step 4: Create FAISS vector store and add embeddings
faiss_store = FAISS.from_texts(chunks, sentence_embedding_model)
print("Embeddings created and stored in FAISS index.")

# Step 5: Set up Hugging Face LLM using a pipeline (e.g., Falcon-7B model)
generator = pipeline("text-generation", model="tiiuae/falcon-7b-instruct", max_length=200, do_sample=True, temperature=0.7)
llm = HuggingFacePipeline(pipeline=generator)

# Step 6: Set up the RetrievalQA chain
retriever = faiss_store.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Step 7: Query the chain
query = "What topics are covered in the syllabus?"
result = qa_chain.run(query)
print("Answer:", result)


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
documents = loader.load_and_split(text_splitter)
